In [1]:
import torch
import numpy as np
from datatypes import torchcomplex, show
from gate_implementation import apply_gate, apply_sparse_gate, device
import time

In [2]:
def test_correctness():
    psi0 = torch.arange(8, device=device) + 1
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    XZ = np.kron(X, Z)
    XZ = torchcomplex(XZ).to(device)
    psi1 = apply_gate([1, 2], XZ, psi0)
    np.testing.assert_allclose(
        psi1.cpu(), torch.tensor([3, -4, 1, -2, 7, -8, 5, -6]).cpu(), atol=0.00001
    )
    print("Correctness test passed for dense")

    XZ_sparse=(
        [(0,2),(1,3),(2,0),(3,1)],
        torchcomplex(np.array([1.0,-1.0,1.0,-1.0]))
    )
    psi2 = apply_sparse_gate([1, 2], XZ_sparse, psi0)
    np.testing.assert_allclose(
        psi2.cpu(), torch.tensor([3, -4, 1, -2, 7, -8, 5, -6]).cpu(), atol=0.00001
    )
    print("Correctness test passed for sparse")

In [3]:
class PerfTest:
    @staticmethod
    def time(n, positions, sparse=False):

        if device.type == "cuda":
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)

        if isinstance(positions, int):
            positions = list(range(positions))

        k = len(positions)
        if sparse:
            matrix = PerfTest.get_k_qubit_sparse_Xk(k)
        else:
            matrix = PerfTest.get_k_qubit_dense_Hadamard(k)

        psi0 = PerfTest.get_state(n)
        if device.type == "cuda":
            start.record()
            PerfTest.apply_gate_to_qubits(positions, matrix, psi0, sparse=sparse)
            end.record()
            """https://discuss.pytorch.org/t/how-to-measure-time-in-pytorch/26964/10"""
            torch.cuda.synchronize()

        else:
            t0 = time.time()
            PerfTest.apply_gate_to_qubits(positions, matrix, psi0, sparse=sparse)
            t1 = time.time()

        if device.type == "cuda":
            s = start.elapsed_time(end) / 1000
        else:
            s = t1 - t0
            
        if sparse:
            print(f"{s:.4f} s for sparse {k} qubit X^k gate on {n} qubit state")
        else:
            print(f"{s:.4f} s for dense {k} qubit Hadamard gate on {n} qubit state")

        return s

    @staticmethod
    def apply_gate_to_qubits(positions, matrix, psi0, sparse=False):
        if sparse:
            return apply_sparse_gate(positions, matrix, psi0)
        else:
            return apply_gate(positions, matrix, psi0)
        
    @staticmethod
    def get_state(n):
        return torch.arange(2**n, device=device)
    
    @staticmethod
    def get_k_qubit_dense_Hadamard(k):
        X = torchcomplex(np.array([[1, 1], [1, -1]]) / np.sqrt(2)).to(device)
        X_ = torchcomplex(np.array([[1]])).to(device)
        for _ in range(k):
            X_ = torch.kron(X_, X)
        return X_
    
    @staticmethod
    def get_k_qubit_sparse_Xk(k):
        K = 2**k
        I = torch.arange(K, device=device)
        J = K - I
        V = torch.ones(K, device=device)
        matrix_indices = torch.stack([I, J], dim=1)
        return (matrix_indices, V)

In [4]:
test_correctness()

Correctness test passed for dense
Correctness test passed for sparse


/home/nilin/diffcircuit/datatypes.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  imag = torch.Tensor(x.imag)


In [5]:
densetimes=dict()
sparsetimes=dict()

for k in range(1,12):
    densetimes[k]=PerfTest.time(20, k)

for k in range(1,19):
    sparsetimes[k]=PerfTest.time(20, k, sparse=True)

0.0020 s for dense 1 qubit Hadamard gate on 20 qubit state
0.0047 s for dense 2 qubit Hadamard gate on 20 qubit state
0.0117 s for dense 3 qubit Hadamard gate on 20 qubit state
0.0305 s for dense 4 qubit Hadamard gate on 20 qubit state
0.0822 s for dense 5 qubit Hadamard gate on 20 qubit state
0.2441 s for dense 6 qubit Hadamard gate on 20 qubit state
0.8277 s for dense 7 qubit Hadamard gate on 20 qubit state
3.0764 s for dense 8 qubit Hadamard gate on 20 qubit state
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/nilin/anaconda3/envs/lbg/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_59125/2539455613.py", line 5, in <module>
    densetimes[k]=PerfTest.time(20, k)
  File "/tmp/ipykernel_59125/1094434889.py", line 21, in time
    PerfTest.apply_gate_to_qubits(positions, matrix, psi0, sparse=sparse)
  File "/tmp/ipykernel_59125/1094434889.py", line 48, in apply_gate_to_qubits
    return apply_gate(positions, matrix, psi0)
  File "/home/nilin/diffcircuit/gate_implementation.py", line -1, in apply_gate
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nilin/anaconda3/envs/lbg/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/nilin/anaconda3/envs/

In [ ]:
densetimes[11]=PerfTest.time(20,11)

In [ ]:
import matplotlib.pyplot as plt


plt.xlabel("k")
plt.ylabel("Time (s)")
plt.yscale("log")
plt.axhline(60,lw=.5,ls=':',color='gray',label="1 min")
plt.axhline(1,lw=.5,color='gray',label="1 second")
plt.axvline(10,lw=.5,ls=':',color='gray')
plt.xticks(list(range(1,21)))

plt.plot(densetimes.keys(), densetimes.values(),'-o', label="dense Hadamard gate H^k")
plt.plot(sparsetimes.keys(), sparsetimes.values(),'--s', label="sparse X^k gate")

plt.legend()

plt.title("Time to apply k-qubit gate to 20-qubit state")

In [ ]:
#plt.savefig("example_outputs/many_qubit_gate_perftest.pdf")